In [1]:
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [2]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'当前设备为{device}')

当前设备为cuda:0


In [4]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn



rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)


voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  
input_size = 300 
#dataset
train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoader
batch_size = 6

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)


dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


In [5]:
from utils.ssd_model import SSD


ssd_cfg = {
    'num_classes': 21, 
    'input_size': 300, 
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],
    'feature_maps': [38, 19, 10, 5, 3, 1],  
    'steps': [8, 16, 32, 64, 100, 300],  
    'min_sizes': [30, 60, 111, 162, 213, 264],  
    'max_sizes': [60, 111, 162, 213, 264, 315],  
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}


net = SSD(phase="train", cfg=ssd_cfg)


vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)




def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None: 
            nn.init.constant_(m.bias, 0.0)



net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用设备为", device)

print('网络设置完毕，学习的权重也加载完成')


使用设备为 cuda:0
网络设置完毕，学习的权重也加载完成


In [6]:
from utils.ssd_model import MultiBoxLoss


criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)


optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)

In [7]:
#创建模型


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用设备", device)


    net.to(device)

    # 当网络在一定程度稳定下来时开启高速运算
    torch.backends.cudnn.benchmark = True

    iteration = 1
    epoch_train_loss = 0.0  
    epoch_val_loss = 0.0 
    logs = []


    for epoch in range(num_epochs+1):


        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('轮数 {}/{}'.format(epoch+1, num_epochs))
        print('-------------')


        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval() 
                    print('-------------')
                    print('（val）')
                else:

                    continue


            for images, targets in dataloaders_dict[phase]:


                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]


                optimizer.zero_grad()


                with torch.set_grad_enabled(phase == 'train'):

                    outputs = net(images)

                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c


                    if phase == 'train':
                        loss.backward()

                     
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()

                        if (iteration % 10 == 0):
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('迭代中 {} || 损失: {:.4f} || 10次迭代: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()


        t_epoch_finish = time.time()
        print('-------------')
        print('轮 {} || Epoch_训练损失:{:.4f} ||Epoch_测试损失:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()


        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0 
        epoch_val_loss = 0.0 


        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')


In [8]:
num_epochs= 1
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


使用设备 cuda:0
-------------
轮数 1/1
-------------
（train）


C:\Users\Administrator\SSD_object_detection\utils\data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


迭代中 10 || 损失: 15.8498 || 10次迭代: 11.5787 sec.
迭代中 20 || 损失: 16.2861 || 10次迭代: 4.7393 sec.
迭代中 30 || 损失: 12.0303 || 10次迭代: 4.7558 sec.
迭代中 40 || 损失: 11.1153 || 10次迭代: 4.6213 sec.
迭代中 50 || 损失: 12.6515 || 10次迭代: 4.6825 sec.
迭代中 60 || 损失: 9.3104 || 10次迭代: 4.5508 sec.
迭代中 70 || 损失: 9.7286 || 10次迭代: 4.6951 sec.
迭代中 80 || 损失: 8.1316 || 10次迭代: 4.8813 sec.
迭代中 90 || 损失: 10.5633 || 10次迭代: 4.6786 sec.
迭代中 100 || 损失: 9.9221 || 10次迭代: 4.7664 sec.
迭代中 110 || 损失: 9.7299 || 10次迭代: 5.0808 sec.
迭代中 120 || 损失: 9.0026 || 10次迭代: 4.7954 sec.
迭代中 130 || 损失: 8.5835 || 10次迭代: 5.0206 sec.
迭代中 140 || 损失: 10.2168 || 10次迭代: 5.0497 sec.
迭代中 150 || 损失: 9.1491 || 10次迭代: 4.6602 sec.
迭代中 160 || 损失: 8.1679 || 10次迭代: 5.0231 sec.
迭代中 170 || 损失: 8.1633 || 10次迭代: 4.8137 sec.
迭代中 180 || 损失: 8.9868 || 10次迭代: 4.7568 sec.
迭代中 190 || 损失: 8.1548 || 10次迭代: 4.8076 sec.
迭代中 200 || 损失: 9.2872 || 10次迭代: 5.1040 sec.
迭代中 210 || 损失: 9.1189 || 10次迭代: 5.1134 sec.
迭代中 220 || 损失: 8.3029 || 10次迭代: 5.1419 sec.
迭代中 230 || 损失: 8.7552 || 10次迭代: 5

迭代中 1830 || 损失: 8.5794 || 10次迭代: 5.0208 sec.
迭代中 1840 || 损失: 5.7758 || 10次迭代: 4.9335 sec.
迭代中 1850 || 损失: 7.4043 || 10次迭代: 5.1143 sec.
迭代中 1860 || 损失: 6.4769 || 10次迭代: 5.0922 sec.
迭代中 1870 || 损失: 6.2332 || 10次迭代: 5.2038 sec.
迭代中 1880 || 损失: 6.6885 || 10次迭代: 4.7933 sec.
迭代中 1890 || 损失: 6.5419 || 10次迭代: 4.6957 sec.
迭代中 1900 || 损失: 6.4178 || 10次迭代: 4.7635 sec.
-------------
轮 2 || Epoch_训练损失:6505.2705 ||Epoch_测试损失:0.0000
timer:  482.4275 sec.


In [9]:
torch.save(net.state_dict() , "weights/ssd300_50.pth")